In [34]:
#Installing packages
library(data.table)
library(tidyverse)
library(stringi)
library(tidyr)

In [90]:
survey_data <- read.csv("C:\\Users\\joyse\\Downloads\\comps_subs_conjoint_v2_March+1,+2024_10.34 (1)\\comps_subs_conjoint_v2_March 1, 2024_10.34.csv")
comp_data <-read.csv("C:\\Users\\joyse\\Dropbox (MIT)\\survey\\input_data\\comps_triplets_10p_v1.csv")
sub_data <-read.csv("C:\\Users\\joyse\\Dropbox (MIT)\\survey\\input_data\\subs_triplets_10p_v1.csv")

# Cleaning Survey Data

In [91]:
head(survey_data)

,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,RecipientLastName,⋯,X1_Q8.1,X2_Q8.1,X3_Q8.1,X4_Q8.1,X5_Q8.1,X6_Q8.1,X7_Q8.1,X8_Q8.1,X9_Q8.1,X10_Q8.1
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,⋯,"Freshburst Listerine Antiseptic Mouthwash For Bad Breath, 1.5 L - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Kashi Heart to Heart Oat Cereal, Organic Blueberry Clusters, 13.4 Oz - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Kleenex Slim Pack Facial Tissues, White, 10 Sheets/Box, 2 Ct - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Folgers Instant Coffee, Classic Roast, 8 Oz, 1 Ct - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Earthborn Holistic Grain-Free Chicken Catcciatori Natural Wet Cat Food, 5.5 Oz - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Hint Water, Watermelon, 16 Fl Oz - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Acana Appalachian Ranch Grain-Free Red Meat & Catfish Dry Dog Food, 25 lb - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Vaseline Baby Petroleum Jelly 13 oz - If [Field-1] is unavailable, which of the following products would you consider buying instead?","McCafÃ© Medium Roast Decaf Ground Coffee, Premium, 12 Oz - If [Field-1] is unavailable, which of the following products would you consider buying instead?","Clarisse Cotton Swabs, Plastic Stick, 375 ct. - If [Field-1] is unavailable, which of the following products would you consider buying instead?"
2,"{""ImportId"":""startDate"",""timeZone"":""America/New_York""}","{""ImportId"":""endDate"",""timeZone"":""America/New_York""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America/New_York""}","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",⋯,"{""ImportId"":""1_QID6""}","{""ImportId"":""2_QID6""}","{""ImportId"":""3_QID6""}","{""ImportId"":""4_QID6""}","{""ImportId"":""5_QID6""}","{""ImportId"":""6_QID6""}","{""ImportId"":""7_QID6""}","{""ImportId"":""8_QID6""}","{""ImportId"":""9_QID6""}","{""ImportId"":""10_QID6""}"
3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,${lm://Field/3},${lm://Field/2},${lm://Field/3},${lm://Field/3},${lm://Field/2},${lm://Field/3},${lm://Field/2},${lm://Field/3},${lm://Field/2},${lm://Field/3}
4,2024-02-27 15:40:30,2024-02-27 15:41:32,Survey Preview,,100,62,True,2024-02-27 15:41:33,R_1tGbTcv9Opeo7Uw,,⋯,,,,,,,,,,
5,2024-02-27 16:06:21,2024-02-27 16:06:32,Survey Preview,,100,11,True,2024-02-27 16:06:33,R_1golRVgEoYUAkaE,,⋯,,,,,,,,,,
6,2024-02-27 16:06:40,2024-02-27 16:07:03,Survey Preview,,100,22,True,2024-02-27 16:07:04,R_138dnQn0uJpiUy1,,⋯,,,,,,,,,,


In [92]:
cols_to_pivot <- grep("Q5|Q8", names(survey_data), value = TRUE)

# Pivot the dataset from wide to long format
survey_data_filtered <- subset(survey_data, grepl("^R_", ResponseId)) #Only do this for valid respondents

survey_data_long <- pivot_longer(survey_data_filtered, 
                                 cols = cols_to_pivot,       # Columns to pivot
                                 names_to = "Question",      # Name of the column containing question names
                                 values_to = "Question_content" # Name of the column containing response values  
)

#Get rid of rows where the Question wasn't answered by the respondent
survey_data_long <- survey_data_long %>%
                     filter(Question_content != '')


# Print the first few rows of the long format dataset
head(survey_data_long)

StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,RecipientLastName,⋯,Q1.2,Q1.3,Q2.1,Q2.2,Q2.3,Q2.4,Q3.2,Q6.2,Question,Question_content
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,,,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X1_Q5.1,${lm://Field/3}
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,,,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X2_Q5.1,${lm://Field/2}
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,,,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X3_Q5.1,${lm://Field/2}
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,,,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X4_Q5.1,${lm://Field/2}
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,,,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X9_Q5.1,${lm://Field/2}
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,,,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X1_Q8.1,${lm://Field/3}


In [93]:
#Create indicators for if the question was a complimentary or substitute question
survey_data_long <- survey_data_long %>%
                     mutate(Comp_question = ifelse(grepl("Q5", Question), 1, 0))
survey_data_long <- survey_data_long %>%
                     mutate(Sub_question = ifelse(grepl("Q8", Question), 1, 0))

head(survey_data_long)

StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,RecipientLastName,⋯,Q2.1,Q2.2,Q2.3,Q2.4,Q3.2,Q6.2,Question,Question_content,Comp_question,Sub_question
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X1_Q5.1,${lm://Field/3},1,0
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X2_Q5.1,${lm://Field/2},1,0
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X3_Q5.1,${lm://Field/2},1,0
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X4_Q5.1,${lm://Field/2},1,0
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X9_Q5.1,${lm://Field/2},1,0
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Female,Asian / Pacific Islander,18 to 24,No schooling completed,,,X1_Q8.1,${lm://Field/3},0,1


In [94]:
#Create a question_number column for later merging 
survey_data_long <- survey_data_long %>%
                     mutate(Question_Number = as.numeric(str_extract(Question, "(?<=X)\\d+")))


# Print the first few rows of the updated dataset
head(survey_data_long)

StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,RecipientLastName,⋯,Q2.2,Q2.3,Q2.4,Q3.2,Q6.2,Question,Question_content,Comp_question,Sub_question,Question_Number
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Asian / Pacific Islander,18 to 24,No schooling completed,,,X1_Q5.1,${lm://Field/3},1,0,1
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Asian / Pacific Islander,18 to 24,No schooling completed,,,X2_Q5.1,${lm://Field/2},1,0,2
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Asian / Pacific Islander,18 to 24,No schooling completed,,,X3_Q5.1,${lm://Field/2},1,0,3
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Asian / Pacific Islander,18 to 24,No schooling completed,,,X4_Q5.1,${lm://Field/2},1,0,4
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Asian / Pacific Islander,18 to 24,No schooling completed,,,X9_Q5.1,${lm://Field/2},1,0,9
2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,,⋯,Asian / Pacific Islander,18 to 24,No schooling completed,,,X1_Q8.1,${lm://Field/3},0,1,1


# Cleaning Comp and Sub Data

In [95]:
comp_data <- comp_data %>%
                     mutate(index = row_number())
sub_data <- sub_data %>%
                     mutate(index = row_number())

head(comp_data)
head(sub_data)

,sku_A,sku_B,cos_sim_B,sku_title_A,l0_A,l1_A,l2_A,sku_title_B,l0_B,l1_B,l2_B,cos_sim_dec_B,sku_C,cos_sim_C,sku_title_C,l0_C,l1_C,l2_C,cos_sim_dec_C,index
,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,165,10594,0.3419732,"Freshburst Listerine Antiseptic Mouthwash For Bad Breath, 1.5 L",Health & Beauty,Personal Care,Mouthwash,"Gold Bond Ultimate Eczema Relief Skin Protectant Lotion 2%, Colloidal Oatmeal, 14 Oz",Health & Beauty,Skin Care,Body Moisturizers,1,33885,0.3681883,"Old Spice 2-in-1 Shampoo & Conditioner, Pure Sport, 25.3 Oz",Health & Beauty,Hair Care Products,Shampoos,4,1
2,502,7403,0.2694587,"Kashi Heart to Heart Oat Cereal, Organic Blueberry Clusters, 13.4 Oz",Grocery,Breakfast Foods,Cold Cereals,"Dr. McDougall's Right Foods Soup, Chili Bean, 18 Ounce",Grocery,"Soups, Meals & Side Dishes",Chilis & Stews,5,8372,0.2708866,Pop-Secret Sea Salt Microwaveable Popcorn 6ct 19.2oz,Grocery,Snack Foods,Popcorn & Puffed Snacks,5,2
3,1520,5612,0.3317442,"Kleenex Slim Pack Facial Tissues, White, 10 Sheets/Box, 2 Ct",Household Supplies,Paper & Plastic Products,Facial Tissues,"Seventh Generation Free & Clear Baby Wipes, Flip Top, Unscented, 6 Packs of 64 (384 ct)",Baby,Diapering,Baby Wipes,9,22414,0.3116748,"Clinique Moisture Surge Extra Thirsty Skin Relief, 1.7 Oz",Health & Beauty,Skin Care,Facial Moisturizers & Treatments,8,3
4,1740,19946,0.2674786,"Folgers Instant Coffee, Classic Roast, 8 Oz, 1 Ct",Grocery,Beverages,Coffee,"MOLINA VANILLA, EXTRACT VANILLA ORGNL, 8.3 OZ",Grocery,Cooking & Baking Supplies,Extracts,3,27774,0.2790355,"Double """"Q"""" Pink Wild Caught Alaskan Salmon, 14.75 Oz",Grocery,Canned & Jarred Food,Canned Meat & Seafood,4,4
5,6215,14839,0.3410132,"Earthborn Holistic Grain-Free Chicken Catcciatori Natural Wet Cat Food, 5.5 Oz",Pet Supplies,Cat Supplies,Cat Food,"Stella & Chewy's Dinner Morsels Grain-Free Chicken Freeze Dried Cat Food, 18 oz",Pet Supplies,Cat Supplies,Cat Food,7,25629,0.3744867,"Blue Buffalo Healthy Gourmet Ocean Fish & Tuna Pate Recipe Wet Cat Food, 3 Oz",Pet Supplies,Cat Supplies,Cat Food,8,5
6,6617,19570,0.3718903,"Hint Water, Watermelon, 16 Fl Oz",Grocery,Beverages,Water,"Firehook Mediterranean Baked Crackers, Garlic & Thyme, 7 Oz",Grocery,Snack Foods,Bread & Crackers,10,21267,0.2274641,"Laffy Taffy Rope Strawberry, .81 Oz each, 24 Ct",Grocery,"Candy, Gum & Chocolate",Chewy Candy,4,6


,sku_A,sku_B,cos_sim_B,sku_title_A,l0_A,l1_A,l2_A,sku_title_B,l0_B,l1_B,l2_B,cos_sim_dec_B,sku_C,cos_sim_C,sku_title_C,l0_C,l1_C,l2_C,cos_sim_dec_C,index
,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,165,33321,0.3146814,"Freshburst Listerine Antiseptic Mouthwash For Bad Breath, 1.5 L",Health & Beauty,Personal Care,Mouthwash,"The Art of Shaving Large Fragrance Gift Set, Oud, 2 Ct (Value of $155)",Health & Beauty,Fragrance,Men's Fragrance,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,1
2,502,10274,0.5617551,"Kashi Heart to Heart Oat Cereal, Organic Blueberry Clusters, 13.4 Oz",Grocery,Breakfast Foods,Cold Cereals,"Honey Nut Cheerios Cereal, Medley Crunch, 13.1 Oz",Grocery,Breakfast Foods,Cold Cereals,8,34484,0.3104984,"Chilli Man Vegetarian Chili with Beans, 15 oz, (Pack of 12)",Grocery,"Soups, Meals & Side Dishes",Chilis & Stews,2,2
3,1520,15148,0.3108335,"Kleenex Slim Pack Facial Tissues, White, 10 Sheets/Box, 2 Ct",Household Supplies,Paper & Plastic Products,Facial Tissues,"Eucerin Original Moisturizing Lotion, 1 Oz",Health & Beauty,Skin Care,Body Moisturizers,6,4620,0.3140773,"Quilted Northern Ultra Plush Toilet Paper, 24 Double Rolls",Household Supplies,Paper & Plastic Products,Toilet Paper,6,3
4,1740,1868,0.6909340,"Folgers Instant Coffee, Classic Roast, 8 Oz, 1 Ct",Grocery,Beverages,Coffee,"Folgers Gourmet Selections Medium Roast Single Serve for Keurig, Classic Roast, 12 Ct",Grocery,Beverages,Coffee,10,27088,0.3721115,Lipton Mango Unsweetened Iced Tea Mix 10 qt,Grocery,Beverages,Drink Mixes & Syrups,5,4
5,6215,7849,0.4628579,"Earthborn Holistic Grain-Free Chicken Catcciatori Natural Wet Cat Food, 5.5 Oz",Pet Supplies,Cat Supplies,Cat Food,"Purina Friskies Seafood Sensations Dry Cat Food, 16 Lb.",Pet Supplies,Cat Supplies,Cat Food,4,16055,0.6279207,"Purina Pro Plan FOCUS Adult Indoor Care Salmon & Rice Entree in Sauce Wet Cat Food, 3 Oz. (Pack of 24)",Pet Supplies,Cat Supplies,Cat Food,8,5
6,6617,7462,0.4637846,"Hint Water, Watermelon, 16 Fl Oz",Grocery,Beverages,Water,"Dasani Purified Sparkling Water, Strawberry Guava, 12 Fl Oz, 8 Ct",Grocery,Beverages,Water,9,7400,0.3822151,"Lipton Diet Green Tea, Citrus, 16.9 Fl Oz (Innerpack of 12)",Grocery,Beverages,Tea,8,6


# Merging Comp and Sub Data with Survey Data

In [96]:
#Merge comp_data with survey_long
merged_data <- merge(survey_data_long,comp_data, by.x = "Question_Number", by.y = "index", all.x = TRUE)

#Sorting by response ID
merged_data <- merged_data %>% arrange(ResponseId)

#Adding prefix of comp to all columns from comp_data
comp_columns <- names(merged_data)[names(merged_data) %in% names(comp_data)]
names(merged_data)[names(merged_data) %in% comp_columns] <- paste0("comp_", comp_columns)


# Print the merged dataset
head(merged_data)

,Question_Number,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,⋯,comp_l1_B,comp_l2_B,comp_cos_sim_dec_B,comp_sku_C,comp_cos_sim_C,comp_sku_title_C,comp_l0_C,comp_l1_C,comp_l2_C,comp_cos_sim_dec_C
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
1,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Skin Care,Body Moisturizers,1,33885,0.3681883,"Old Spice 2-in-1 Shampoo & Conditioner, Pure Sport, 25.3 Oz",Health & Beauty,Hair Care Products,Shampoos,4
2,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Skin Care,Body Moisturizers,1,33885,0.3681883,"Old Spice 2-in-1 Shampoo & Conditioner, Pure Sport, 25.3 Oz",Health & Beauty,Hair Care Products,Shampoos,4
3,2,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,"Soups, Meals & Side Dishes",Chilis & Stews,5,8372,0.2708866,Pop-Secret Sea Salt Microwaveable Popcorn 6ct 19.2oz,Grocery,Snack Foods,Popcorn & Puffed Snacks,5
4,2,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,"Soups, Meals & Side Dishes",Chilis & Stews,5,8372,0.2708866,Pop-Secret Sea Salt Microwaveable Popcorn 6ct 19.2oz,Grocery,Snack Foods,Popcorn & Puffed Snacks,5
5,3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Diapering,Baby Wipes,9,22414,0.3116748,"Clinique Moisture Surge Extra Thirsty Skin Relief, 1.7 Oz",Health & Beauty,Skin Care,Facial Moisturizers & Treatments,8
6,3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Diapering,Baby Wipes,9,22414,0.3116748,"Clinique Moisture Surge Extra Thirsty Skin Relief, 1.7 Oz",Health & Beauty,Skin Care,Facial Moisturizers & Treatments,8


In [97]:
#Merge sub_data with merged_data
merged_data <- merge(merged_data,sub_data, by.x = "Question_Number", by.y = "index", all.x = TRUE)

#Sorting by response ID
merged_data <- merged_data %>% arrange(ResponseId, Sub_question)

#Adding a prefix of sub to all columns from sub data
sub_columns <- names(merged_data)[names(merged_data) %in% names(sub_data)]
names(merged_data)[names(merged_data) %in% sub_columns] <- paste0("sub_", sub_columns)


# Print the merged dataset
head(merged_data)
#At this point, all of the data is merged

,Question_Number,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,⋯,sub_l1_B,sub_l2_B,sub_cos_sim_dec_B,sub_sku_C,sub_cos_sim_C,sub_sku_title_C,sub_l0_C,sub_l1_C,sub_l2_C,sub_cos_sim_dec_C
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
1,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Fragrance,Men's Fragrance,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5
2,2,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Breakfast Foods,Cold Cereals,8,34484,0.3104984,"Chilli Man Vegetarian Chili with Beans, 15 oz, (Pack of 12)",Grocery,"Soups, Meals & Side Dishes",Chilis & Stews,2
3,3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Skin Care,Body Moisturizers,6,4620,0.3140773,"Quilted Northern Ultra Plush Toilet Paper, 24 Double Rolls",Household Supplies,Paper & Plastic Products,Toilet Paper,6
4,4,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Beverages,Coffee,10,27088,0.3721115,Lipton Mango Unsweetened Iced Tea Mix 10 qt,Grocery,Beverages,Drink Mixes & Syrups,5
5,9,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Paper & Plastic Products,Disposable Coffee Filters,7,22246,0.6978610,"Folgers Decaf Ground Coffee, Original, 0.9 Oz, 36 Ct",Grocery,Beverages,Coffee,10
6,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Fragrance,Men's Fragrance,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5


# Creating Result Columns

In [98]:
#Create cos_sim_dif column that measures the cos_sim_dif between product B-product C, depending on comp or sub
merged_data$cos_sim_dif <- ifelse(merged_data$Comp_question == 1, 
                                  merged_data$comp_cos_sim_B - merged_data$comp_cos_sim_C,
                                  merged_data$sub_cos_sim_B - merged_data$sub_cos_sim_C)

# Print the first few rows of the updated dataset
head(merged_data["Question"])
head(merged_data)

,Question
,<chr>
1,X1_Q5.1
2,X2_Q5.1
3,X3_Q5.1
4,X4_Q5.1
5,X9_Q5.1
6,X1_Q8.1


,Question_Number,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,⋯,sub_l2_B,sub_cos_sim_dec_B,sub_sku_C,sub_cos_sim_C,sub_sku_title_C,sub_l0_C,sub_l1_C,sub_l2_C,sub_cos_sim_dec_C,cos_sim_dif
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
1,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Men's Fragrance,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,-0.026215151
2,2,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Cold Cereals,8,34484,0.3104984,"Chilli Man Vegetarian Chili with Beans, 15 oz, (Pack of 12)",Grocery,"Soups, Meals & Side Dishes",Chilis & Stews,2,-0.001427876
3,3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Body Moisturizers,6,4620,0.3140773,"Quilted Northern Ultra Plush Toilet Paper, 24 Double Rolls",Household Supplies,Paper & Plastic Products,Toilet Paper,6,0.020069370
4,4,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Coffee,10,27088,0.3721115,Lipton Mango Unsweetened Iced Tea Mix 10 qt,Grocery,Beverages,Drink Mixes & Syrups,5,-0.011556959
5,9,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Disposable Coffee Filters,7,22246,0.6978610,"Folgers Decaf Ground Coffee, Original, 0.9 Oz, 36 Ct",Grocery,Beverages,Coffee,10,0.050666812
6,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,Men's Fragrance,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,-0.028203775


In [99]:
#Create cos_sim_bin that is 1 if the cos_sim score of Product B is greater than Product C, 0 otherwise 
merged_data$cos_sim_bin <- ifelse(merged_data$cos_sim_dif >= 0, 1, 0)

# Print the first few rows of the updated dataset
head(merged_data)

,Question_Number,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,⋯,sub_cos_sim_dec_B,sub_sku_C,sub_cos_sim_C,sub_sku_title_C,sub_l0_C,sub_l1_C,sub_l2_C,sub_cos_sim_dec_C,cos_sim_dif,cos_sim_bin
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>
1,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,-0.026215151,0
2,2,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,8,34484,0.3104984,"Chilli Man Vegetarian Chili with Beans, 15 oz, (Pack of 12)",Grocery,"Soups, Meals & Side Dishes",Chilis & Stews,2,-0.001427876,0
3,3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,6,4620,0.3140773,"Quilted Northern Ultra Plush Toilet Paper, 24 Double Rolls",Household Supplies,Paper & Plastic Products,Toilet Paper,6,0.020069370,1
4,4,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,10,27088,0.3721115,Lipton Mango Unsweetened Iced Tea Mix 10 qt,Grocery,Beverages,Drink Mixes & Syrups,5,-0.011556959,0
5,9,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,7,22246,0.6978610,"Folgers Decaf Ground Coffee, Original, 0.9 Oz, 36 Ct",Grocery,Beverages,Coffee,10,0.050666812,1
6,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,2,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,-0.028203775,0


In [100]:
#Create the Choice column based on the conditions, 1 if choose correctly, 0 otherwise
merged_data$Choice <- ifelse(merged_data$Question_content == '${lm://Field/2}' & merged_data$cos_sim_bin == 1, 1,
                             ifelse(merged_data$Question_content == '${lm://Field/3}' & merged_data$cos_sim_bin == 0, 1,
                                    0))

# Print the first few rows of the updated dataset
head(merged_data)

,Question_Number,StartDate,EndDate,Status,IPAddress,Progress,Duration..in.seconds.,Finished,RecordedDate,ResponseId,⋯,sub_sku_C,sub_cos_sim_C,sub_sku_title_C,sub_l0_C,sub_l1_C,sub_l2_C,sub_cos_sim_dec_C,cos_sim_dif,cos_sim_bin,Choice
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
1,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,-0.026215151,0,1
2,2,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,34484,0.3104984,"Chilli Man Vegetarian Chili with Beans, 15 oz, (Pack of 12)",Grocery,"Soups, Meals & Side Dishes",Chilis & Stews,2,-0.001427876,0,0
3,3,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,4620,0.3140773,"Quilted Northern Ultra Plush Toilet Paper, 24 Double Rolls",Household Supplies,Paper & Plastic Products,Toilet Paper,6,0.020069370,1,1
4,4,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,27088,0.3721115,Lipton Mango Unsweetened Iced Tea Mix 10 qt,Grocery,Beverages,Drink Mixes & Syrups,5,-0.011556959,0,0
5,9,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,22246,0.6978610,"Folgers Decaf Ground Coffee, Original, 0.9 Oz, 36 Ct",Grocery,Beverages,Coffee,10,0.050666812,1,1
6,1,2024-02-27 15:17:11,2024-02-27 15:18:15,Survey Preview,,100,63,True,2024-02-27 15:18:16,R_1O6ohqQ0k7PbHuS,⋯,14181,0.3428852,"Johnson's Buddies No More Tangles Hair Detangler For Kids, 10 Fl. Oz.",Health & Beauty,Hair Care Products,Styling Products,5,-0.028203775,0,1


In [101]:
#Export as CSV
write.csv(merged_data, file = "C:\\Users\\joyse\\Desktop\\UROP\\merged_data.csv", row.names = FALSE)